In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter, Venus, Mars 
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
## Main dates (1 набор дат)
date_launch = Time("2026-08-07", scale="utc").tdb
date_flyby_Venus = Time("2028-05-23", scale="utc").tdb
date_flyby_Earth = Time("2030-02-05", scale="utc").tdb
date_flyby_Mars = Time("2032-05-03", scale="utc").tdb
date_arrival = Time("2034-01-24", scale="utc").tdb

In [3]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_launch))
ven = Ephem.from_body(Venus, time_range(date_launch, end=date_flyby_Venus))
earth_2 = Ephem.from_body(Earth, time_range(date_launch, end=date_flyby_Earth))
mar = Ephem.from_body(Mars, time_range(date_launch, end=date_flyby_Mars))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [4]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_ven = Orbit.from_ephem(Sun, ven, date_flyby_Venus)
ss_earth_2 = Orbit.from_ephem(Sun, earth_2, date_flyby_Earth)
ss_mar = Orbit.from_ephem(Sun, mar, date_flyby_Mars)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [5]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_ven)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_ven, ss_earth_2)
ss_trans_2, ss_target_2 = ss_ven.apply_maneuver(man_lambert_2, intermediate=True)

man_lambert_3 = Maneuver.lambert(ss_earth_2, ss_mar)
ss_trans_3, ss_target_3 = ss_earth_2.apply_maneuver(man_lambert_3, intermediate=True)

man_lambert_4 = Maneuver.lambert(ss_mar, ss_jup)
ss_trans_4, ss_target_4 = ss_mar.apply_maneuver(man_lambert_4, intermediate=True)

In [6]:
#1
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=360 * u.deg), color="blue", label="Transfer orbit E-V"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=280 * u.deg), color="purple", label="Transfer orbit V-E"
)

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=255 * u.deg), color="crimson", label="Transfer orbit E-M"
)

plotter.plot_trajectory(
    ss_trans_4.sample(max_anomaly=161 * u.deg), color="black", label="Transfer orbit M-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [7]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")

plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=360 * u.deg), color="blue", label="Transfer orbit E-V"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [8]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=280 * u.deg), color="purple", label="Transfer orbit V-E"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [9]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=255 * u.deg), color="crimson", label="Transfer orbit E-M"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [10]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")

plotter.plot_trajectory(
    ss_trans_4.sample(max_anomaly=161 * u.deg), color="black", label="Transfer orbit M-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [12]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_ven, v_ven = Ephem.from_body(Venus, date_flyby_Venus).rv(date_flyby_Venus)
r_earth_2, v_earth_2 = Ephem.from_body(Earth, date_flyby_Earth).rv(date_flyby_Earth)
r_mar, v_mar = Ephem.from_body(Mars, date_flyby_Mars).rv(date_flyby_Mars)
r_jup, v_jup = Ephem.from_body(Jupiter, date_arrival).rv(date_arrival)

In [13]:
# Фазовые координаты Земли_1

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

105468142.1213199 km
-100586572.16119222 km
-43587045.298151545 km

[20.90713044 18.93758892  8.20868726] km / s


In [14]:
# Фазовые координаты Венеры

print (f"{r_ven [0]}")
print (f"{r_ven [1]}")
print (f"{r_ven [2]}"'\n')

print (v_ven.to(u.km / u.s))

-60339375.34500821 km
-83729931.19146344 km
-33836853.110404626 km

[ 28.81227061 -17.3108634   -9.61246121] km / s


In [15]:
# Фазовые координаты Земли_1

print (f"{r_earth_2 [0]}")
print (f"{r_earth_2 [1]}")
print (f"{r_earth_2 [2]}"'\n')

print (v_earth_2.to(u.km / u.s))

-105673339.0987906 km
94403113.81799525 km
40927876.710572734 km

[-21.26197118 -19.70126719  -8.53994391] km / s


In [16]:
# Фазовые координаты Марса

print (f"{r_mar [0]}")
print (f"{r_mar [1]}")
print (f"{r_mar [2]}"'\n')

print (v_mar.to(u.km / u.s))

58055690.336583704 km
202219193.7920005 km
91190119.10840175 km

[-22.51834473   7.27452773   3.94380127] km / s


In [17]:
# Фазовые координаты Юпитера

print (f"{r_jup [0]}")
print (f"{r_jup [1]}")
print (f"{r_jup [2]}"'\n')

print (v_jup.to(u.km / u.s))

720610164.8258063 km
-164868949.3459338 km
-88203109.69387917 km

[ 3.10700342 12.21753336  5.16104881] km / s


In [18]:
nh_tof_1 = date_flyby_Venus - date_launch # время полета Земля1 - Венера

(v0_Ea1, vk_Ven), = iod.lambert(Sun.k, r_earth, r_ven, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=655.0000000224517>

In [19]:
# Гиперболический избыток скорости при отлете от Земли
V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 15.26131491 km / s>

In [20]:
V_inf_vec = (v0_Ea1 - v_earth).to(u.km / u.s)
V_inf_vec

<Quantity [11.60364558, -8.81496258, -4.53426693] km / s>

In [21]:
# Гиперболический избыток скорости при подлете к Венере
V_inf_end = (norm(vk_Ven - v_ven)).to(u.km / u.s)
V_inf_end

<Quantity 8.54127113 km / s>

In [22]:
V_inf_end_vec = (vk_Ven - v_ven).to(u.km / u.s)
V_inf_end_vec

<Quantity [ 6.82990874, -5.1196785 , -0.30749175] km / s>

In [23]:
nh_tof_2 = date_flyby_Earth - date_flyby_Venus # время полета Венера - Земля2

(v0_Ven, vk_Ea2), = iod.lambert(Sun.k, r_ven, r_earth_2, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=622.9999999978398>

In [20]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end = (norm(v0_Ven - v_ven)).to(u.km / u.s)
V_inf_end

<Quantity 9.60742451 km / s>

In [21]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_vec = (v0_Ven - v_ven).to(u.km / u.s)
V_inf_end_vec

<Quantity [ 3.6342525 , -8.72410451, -1.72766169] km / s>

In [22]:
# Гиперболический избыток скорости при подлете к Земли_2

V_inf_end_2 = (norm(vk_Ea2 - v_earth_2)).to(u.km / u.s)
V_inf_end_2

<Quantity 15.27611976 km / s>

In [23]:
V_inf_end_2_vec = (vk_Ea2 - v_earth_2).to(u.km / u.s)
V_inf_end_2_vec 

<Quantity [ 10.01746039, -10.82830673,  -3.96964677] km / s>

In [24]:
nh_tof_3 = date_flyby_Mars - date_flyby_Earth # время полета Земля2 - Марс

(V0_Ea2, Vk_Mar), = iod.lambert(Sun.k, r_earth_2, r_mar, nh_tof_3)
nh_tof_3

<TimeDelta object: scale='tdb' format='jd' value=818.0000000065593>

In [25]:
# Гиперболический избыток скорости при подлете к Земли_2

V_inf_end_2_0 = (norm(V0_Ea2 - v_earth_2)).to(u.km / u.s)
V_inf_end_2_0

<Quantity 10.37409151 km / s>

In [26]:
# Гиперболический избыток скорости при подлете к Земли_2

V_inf_end_2_0_vec = (V0_Ea2 - v_earth_2).to(u.km / u.s)
V_inf_end_2_0_vec

<Quantity [ 3.12725865, -8.83359981, -4.45079122] km / s>

In [16]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_3_k = (norm(Vk_Mar - v_mar)).to(u.km / u.s)
V_inf_3_k

<Quantity 14.99814148 km / s>

In [17]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_3_k_vec =(Vk_Mar - v_mar).to(u.km / u.s)
V_inf_3_k_vec

<Quantity [ -2.5303141 , -13.09603434,  -6.85825365] km / s>

In [28]:
nh_tof_4 = date_arrival - date_flyby_Mars # время полета Марс-Юпитер

(V0_Mar, Vk_jup), = iod.lambert(Sun.k, r_mar, r_jup, nh_tof_4)
nh_tof_4

<TimeDelta object: scale='tdb' format='jd' value=630.999999989452>

In [29]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_3_k_0 = (norm(V0_Mar - v_mar)).to(u.km / u.s)
V_inf_3_k_0

<Quantity 24.2065982 km / s>

In [30]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_3_k_0_vec = (V0_Mar - v_mar).to(u.km / u.s)
V_inf_3_k_0_vec

<Quantity [ -2.91899291, -21.79963674, -10.11012932] km / s>

In [31]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_4_k = (norm(Vk_jup - v_jup)).to(u.km / u.s)
V_inf_4_k

<Quantity 10.59600602 km / s>

In [32]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_4_k_vec = (Vk_jup - v_jup).to(u.km / u.s)
V_inf_4_k_vec

<Quantity [ 5.64472661, -8.25177728, -3.51006792] km / s>

In [21]:
time_adv = nh_tof_1 + nh_tof_2 + nh_tof_3 + nh_tof_4
time_adv

<TimeDelta object: scale='tdb' format='jd' value=2727.0000000163027>

In [22]:
scalar_EV = r_earth[0]*r_ven[0] + r_earth[1]*r_ven[1] +r_earth[2]*r_ven[2] 
scalar_VE_2 = r_earth_2[0]*r_ven[0] + r_earth_2[1]*r_ven[1] +r_earth_2[2]*r_ven[2]
scalar_E_2M = r_earth_2[0]*r_mar[0] + r_earth_2[1]*r_mar[1] +r_earth_2[2]*r_mar[2]
scalar_MJ = r_mar[0]*r_jup[0] + r_mar[1]*r_jup[1] +r_mar[2]*r_jup[2]

In [23]:
fi_EV = (scalar_EV/norm(r_earth)/norm(r_ven))
fi_VE2 = (scalar_VE_2/norm(r_earth_2)/norm(r_ven))
fi_E2M = (scalar_E_2M/norm(r_earth_2)/norm(r_mar))
fi_MJ = (scalar_MJ/norm(r_mar)/norm(r_jup))

In [24]:
np.degrees(np.arccos(fi_EV))

<Quantity 77.6526232 deg>

In [25]:
np.degrees(np.arccos(fi_VE2))

<Quantity 100.47701398 deg>

In [26]:
np.degrees(np.arccos(fi_E2M))

<Quantity 60.43449399 deg>

In [27]:
np.degrees(np.arccos(fi_MJ))

<Quantity 89.84809013 deg>

In [28]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 15.261315 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/r)
dV * u.km / u.s

<Quantity 10.91117531 km / s>

In [29]:
## Необходимый импульс для торможения на эллиптическую орбиту Юпитера 
## Не уверен что это нужно

muJ = 126686534
r_p = 670900
r_a = 4000000
V_inf_jup = 7.3951585

print (sqrt ((2 * mu / R + V_inf_jup**2 )))
print (sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p))))

dV_el = sqrt ((2 * mu / R + V_inf_jup**2 )) - sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p)))
dV_el * u.km / u.s

13.266855668656172
17.983782441612995


<Quantity -4.71692677 km / s>

In [30]:
## параметры участка Землия-Венера 

from math import*

to_rad = pi/180

ist_an_1_EV = 79.7185374 * to_rad
ist_an_2_EV = 0.9849344 * to_rad

exs_EV = (norm(r_ven)/norm(r_earth) - 1) / (cos(ist_an_1_EV) - (norm(r_ven)/norm(r_earth) * cos(ist_an_2_EV)))
p_EV = norm(r_earth)*(1+cos(ist_an_1_EV)*exs_EV)
a_EV = p_EV/(1 - exs_EV*exs_EV)

print (f"Эксцентриситет {exs_EV}")
print (f"Фокальный параметр {p_EV}")
print (f"Большая полуось {a_EV}")

Эксцентриситет 0.5342322479937922
Фокальный параметр 166626807.82147726 km
Большая полуось 233176270.12433836 km


In [31]:
## параметры участка Венера-Земля_2 

ist_an_1_VE2 = 20.6956099 * to_rad
ist_an_2_VE2 = 280.9611623 * to_rad

exs_VE2 = (norm(r_earth_2)/norm(r_ven) - 1) / (cos(ist_an_1_VE2) - (norm(r_earth_2)/norm(r_ven) * cos(ist_an_2_VE2)))
p_VE2 = norm(r_ven)*(1+cos(ist_an_1_VE2)*exs_VE2)
a_VE2 = p_VE2/(1 - exs_VE2*exs_VE2)

print (f"Эксцентриситет {exs_VE2}")
print (f"Фокальный параметр {p_VE2}")
print (f"Большая полуось {a_VE2}")

Эксцентриситет 0.5285660304833845
Фокальный параметр 162315464.54171306 km
Большая полуось 225244825.25321794 km


In [32]:
## параметры участка Землия_2-Марс 

ist_an_1_E2M = 317.4312963 * to_rad
ist_an_2_E2M = 256.5647475 * to_rad

exs_E2M = (norm(r_mar)/norm(r_earth_2) - 1) / (cos(ist_an_1_E2M) - (norm(r_mar)/norm(r_earth_2) * cos(ist_an_2_E2M)))
p_E2M = norm(r_earth_2)*(1+cos(ist_an_1_E2M)*exs_EV)
a_E2M = p_E2M/(1 - exs_E2M*exs_E2M)

print (f"Эксцентриситет {exs_E2M}")
print (f"Фокальный параметр {p_E2M}")
print (f"Большая полуось {a_E2M}")

Эксцентриситет 0.5053020891497908
Фокальный параметр 205521897.76597798 km
Большая полуось 275990644.6112483 km


In [33]:
## параметры участка Марс-Юпитер 

ist_an_1_MJ = 251.1432791 * to_rad
ist_an_2_MJ = 161.0656892 * to_rad

exs_MJ = (norm(r_jup)/norm(r_mar) - 1) / (cos(ist_an_1_MJ) - (norm(r_jup)/norm(r_mar) * cos(ist_an_2_MJ)))
p_MJ = norm(r_mar)*(1+cos(ist_an_1_MJ)*exs_MJ)
a_MJ = p_MJ/(1 - exs_MJ*exs_MJ)

print (f"Эксцентриситет {exs_MJ}")
print (f"Фокальный параметр {p_MJ}")
print (f"Большая полуось {a_MJ}")

Эксцентриситет 0.8176305543366369
Фокальный параметр 168705434.1292533 km
Большая полуось 508945617.67581105 km


In [37]:
## Вектор гиперболического избытка скорости после грав. маневра у Венеры

r_pi_ven = 10262.5814
muVen = 324859

V_inf_Ven = 8.5412711
V_inf_end_vec_t_0 = 6.8299087
V_inf_end_vec_t_1 = -5.1196785
V_inf_end_vec_t_2 = -0.30749175

betta_ven = 2*asin(1/(1+((r_pi_ven*V_inf_Ven*V_inf_Ven)/muVen)))
gamma_ven = pi
Vxy_inf_ven = sqrt (V_inf_end_vec_t_0**2 + V_inf_end_vec_t_1**2)

V_grav_X_Ven = V_inf_end_vec_t_0 * cos(betta_ven) - (V_inf_end_vec_t_1*V_inf_Ven)/Vxy_inf_ven * sin(betta_ven)*cos(gamma_ven) - (V_inf_end_vec_t_0*V_inf_end_vec_t_2)/Vxy_inf_ven * sin(betta_ven)*sin(gamma_ven)
V_grav_Y_Ven = V_inf_end_vec_t_1 * cos(betta_ven) + (V_inf_end_vec_t_0*V_inf_Ven)/Vxy_inf_ven * sin(betta_ven)*cos(gamma_ven) - (V_inf_end_vec_t_1*V_inf_end_vec_t_2)/Vxy_inf_ven * sin(betta_ven)*sin(gamma_ven)
V_grav_Z_Ven = V_inf_end_vec_t_2 * cos(betta_ven) + Vxy_inf_ven * sin(betta_ven)*sin(gamma_ven)
V_grav_Ven = sqrt (V_grav_X_Ven**2 + V_grav_Y_Ven**2 + V_grav_Z_Ven**2) 

print (f"betta_ven      {betta_ven*180/pi} deg")
print (f"V_grav_X_Ven   {V_grav_X_Ven * u.km / u.s}")
print (f"V_grav_Y_Ven   {V_grav_Y_Ven * u.km / u.s}")
print (f"V_grav_Z_Ven   {V_grav_Z_Ven * u.km / u.s}")
print (f"V_grav_Ven     {V_grav_Ven * u.km / u.s}")

betta_ven      35.22803227728532 deg
V_grav_X_Ven   2.623987993100371 km / s
V_grav_Y_Ven   -8.124340485340154 km / s
V_grav_Z_Ven   -0.25117856506926284 km / s
V_grav_Ven     8.54127109868445 km / s


In [38]:
## Вектор гиперболического избытка скорости после грав. маневра у Земли_2

r_pi_Ear = 9831.3843
muEar = 398600

V_inf_Ear = 15.27612
V_inf_Ear_vec_t_0 = 10.01746
V_inf_Ear_vec_t_1 = -10.828307
V_inf_Ear_vec_t_2 = -3.9696468

betta_Ear = 2*asin(1/(1+((r_pi_Ear*V_inf_Ear*V_inf_Ear)/muEar)))
gamma_Ear = pi
Vxy_inf_Ear = sqrt (V_inf_Ear_vec_t_0**2 + V_inf_Ear_vec_t_1**2)

V_grav_X_Ear = V_inf_Ear_vec_t_0 * cos(betta_Ear) - (V_inf_Ear_vec_t_1*V_inf_Ear)/Vxy_inf_Ear * sin(betta_Ear)*cos(gamma_Ear) - (V_inf_Ear_vec_t_0*V_inf_Ear_vec_t_2)/Vxy_inf_Ear * sin(betta_Ear)*sin(gamma_Ear)
V_grav_Y_Ear = V_inf_Ear_vec_t_1 * cos(betta_Ear) + (V_inf_Ear_vec_t_0*V_inf_Ear)/Vxy_inf_Ear * sin(betta_Ear)*cos(gamma_Ear) - (V_inf_Ear_vec_t_1*V_inf_Ear_vec_t_2)/Vxy_inf_Ear * sin(betta_Ear)*sin(gamma_Ear)
V_grav_Z_Ear = V_inf_Ear_vec_t_2 * cos(betta_Ear) + Vxy_inf_Ear * sin(betta_Ear)*sin(gamma_Ear)
V_grav_Ear = sqrt (V_grav_X_Ear**2 + V_grav_Y_Ear**2 + V_grav_Z_Ear**2) 

print (f"betta_Ear      {betta_Ear*180/pi} deg")
print (f"V_grav_X_Ear   {V_grav_X_Ear * u.km / u.s}")
print (f"V_grav_Y_Ear   {V_grav_Y_Ear * u.km / u.s}")
print (f"V_grav_Z_Ear   {V_grav_Z_Ear * u.km / u.s}")
print (f"V_grav_Ear     {V_grav_Ear * u.km / u.s}")

betta_Ear      17.024585132644514 deg
V_grav_X_Ear   6.2953663579691455 km / s
V_grav_Y_Ear   -13.391075579402523 km / s
V_grav_Z_Ear   -3.7956937559421036 km / s
V_grav_Ear     15.276119724695084 km / s


In [39]:
## Вектор гиперболического избытка скорости после грав. маневра у Марса

r_pi_Mar = 3735.9168
muMar = 42828

V_inf_Mar = 14.998141
V_inf_Mar_vec_t_0 = -2.5303141
V_inf_Mar_vec_t_1 = -13.096034
V_inf_Mar_vec_t_2 = -6.8582536

betta_Mar = 2*asin(1/(1+((r_pi_Mar*V_inf_Mar*V_inf_Mar)/muMar)))
gamma_Mar = pi
Vxy_inf_Mar = sqrt (V_inf_Mar_vec_t_0**2 + V_inf_Mar_vec_t_1**2)

V_grav_X_Mar = V_inf_Mar_vec_t_0 * cos(betta_Mar) - (V_inf_Mar_vec_t_1*V_inf_Mar)/Vxy_inf_Mar * sin(betta_Mar)*cos(gamma_Mar) - (V_inf_Mar_vec_t_0*V_inf_Mar_vec_t_2)/Vxy_inf_Mar * sin(betta_Mar)*sin(gamma_Mar)
V_grav_Y_Mar = V_inf_Mar_vec_t_1 * cos(betta_Mar) + (V_inf_Mar_vec_t_0*V_inf_Mar)/Vxy_inf_Mar * sin(betta_Mar)*cos(gamma_Mar) - (V_inf_Mar_vec_t_1*V_inf_Mar_vec_t_2)/Vxy_inf_Mar * sin(betta_Mar)*sin(gamma_Mar)
V_grav_Z_Mar = V_inf_Mar_vec_t_2 * cos(betta_Mar) + Vxy_inf_Mar * sin(betta_Mar)*sin(gamma_Mar)
V_grav_Mar = sqrt (V_grav_X_Mar**2 + V_grav_Y_Mar**2 + V_grav_Z_Mar**2) 

print (f"betta_Mar      {betta_Mar*180/pi} deg")
print (f"V_grav_X_Mar   {V_grav_X_Mar * u.km / u.s}")
print (f"V_grav_Y_Mar   {V_grav_Y_Mar * u.km / u.s}")
print (f"V_grav_Z_Mar   {V_grav_Z_Mar * u.km / u.s}")
print (f"V_grav_Mar     {V_grav_Mar * u.km / u.s}")

betta_Mar      5.558930571540198 deg
V_grav_X_Mar   -3.944894679516685 km / s
V_grav_Y_Mar   -12.75883106890939 km / s
V_grav_Z_Mar   -6.825999860221176 km / s
V_grav_Mar     14.998141163797134 km / s
